In [572]:
import yaml
import os
import glob
import json
from datetime import datetime

prj = "Z:/Amite/Amite_LWI/Models/Amite_HMS/Amite_HMS.hms"
prj_dir, prj_file_tail = os.path.split(prj)
prj_name = prj_file_tail.split(".")[0]

with open("Z:/Amite/Amite_LWI/Models/Amite_HMS/Amite_HMS.hms", "r") as f:
     lines = f.readlines()
lines = [s.strip('\n') for s in lines]
# lines

In [573]:
import copy
keyValueList = copy.deepcopy(lines)  

nest_start = 0
nestList = []
for i,v in enumerate(lines):
        if v == 'End:':
                # If not the beginning of the file, skip a blank line (+1) for the start of the subList.
                if len(nestList) > 0:
                        nestList.append(lines[nest_start+1:i])
                else:
                        nestList.append(lines[nest_start:i])
                nest_start = i+1
# nestList



In [574]:
# Get the unique headers [Project, Control, Basin, etc.] as a list.
headers = []
for subList in nestList:
    headers.append(subList[0].split(":")[0])
unique_headers = list(set(headers))
unique_headers


['Control', 'Project', 'Basin', 'Precipitation']

In [575]:
# Create a dictionary based on keys using the unique headers with values of Title, Filename, and Description
kv = {}
kv['Control Files'] = {}
kv['Basin'] = {}
kv['Precipitaion'] = {}
headers_with_same_parsing = ['Control', 'Basin', 'Precipitation']

for subList in nestList:
    header = subList[0].split(":")[0]
    title = subList[0].split(":")[1]
    find_str = 'Description'
    description = [s for s in subList if find_str in s][0].split(":")[1:][0].strip()

    if header == 'Project':
        find_str = 'File Name'
        filename = [s for s in subList if find_str in s][0].split(":")[1:][0].strip()
        kv['Project'] = {
            'Title': title,
            'Project Output DSS File': filename,
            'Description': description
        }
    
    if any(header == i for i in headers_with_same_parsing):
        find_str = 'filename'
        filename = [s for s in subList if find_str in s.lower()][0].split(":")[1:][0].strip()
        kv['Project'][title] = {
            'File Name': filename,
            'Description': description
        }

kv.keys()

dict_keys(['Control Files', 'Basin', 'Precipitaion', 'Project'])

In [576]:
import glob, os
# prj_name = kv['Project']['Title']
# Get project files
prj_files_List = []

for pFile in glob.glob(rf'{prj_dir}/{prj_name}.*'):
        prj_files_List.append(pFile)

prj_files_List

['Z:/Amite/Amite_LWI/Models/Amite_HMS\\Amite_HMS.access',
 'Z:/Amite/Amite_LWI/Models/Amite_HMS\\Amite_HMS.dss',
 'Z:/Amite/Amite_LWI/Models/Amite_HMS\\Amite_HMS.gage',
 'Z:/Amite/Amite_LWI/Models/Amite_HMS\\Amite_HMS.hms',
 'Z:/Amite/Amite_LWI/Models/Amite_HMS\\Amite_HMS.log',
 'Z:/Amite/Amite_LWI/Models/Amite_HMS\\Amite_HMS.out',
 'Z:/Amite/Amite_LWI/Models/Amite_HMS\\Amite_HMS.pdata',
 'Z:/Amite/Amite_LWI/Models/Amite_HMS\\Amite_HMS.run']

In [577]:
# Add application_date from prj_file modified date
modTimeUnix = os.path.getmtime(prj) 
kv['application_date'] = datetime.fromtimestamp(modTimeUnix).strftime('%Y-%m-%d')

# if args.dss, get dss input files
input_dss_dir = r"Z:\Amite\Amite_LWI\Models\Amite_HMS\data"
dss_files_list = []
for pFile in glob.glob(rf'{input_dss_dir}/*.dss'):
        dss_files_list.append(pFile)

# dss_files_list
dss_common_files_input = []
if len(dss_files_list)>0:
        for f in dss_files_list:
                head, tail = os.path.split(f)
                dss_title = tail.split(".")[0]
                dss_common_files_input.append(
                        {
                                "description": "User Added from Input DSS File Directory",
                                "location": f,
                                "source_dataset": None,
                                "title": dss_title
                        },
                )
dss_common_files_input

[{'description': 'User Added from Input DSS File Directory',
  'location': 'Z:\\Amite\\Amite_LWI\\Models\\Amite_HMS\\data\\Amite_HMS_TimeSeries.dss',
  'source_dataset': None,
  'title': 'Amite_HMS_TimeSeries'},
 {'description': 'User Added from Input DSS File Directory',
  'location': 'Z:\\Amite\\Amite_LWI\\Models\\Amite_HMS\\data\\rainfall_stageIV.dss',
  'source_dataset': None,
  'title': 'rainfall_stageIV'},
 {'description': 'User Added from Input DSS File Directory',
  'location': 'Z:\\Amite\\Amite_LWI\\Models\\Amite_HMS\\data\\Specified_Hyetographs.dss',
  'source_dataset': None,
  'title': 'Specified_Hyetographs'},
 {'description': 'User Added from Input DSS File Directory',
  'location': 'Z:\\Amite\\Amite_LWI\\Models\\Amite_HMS\\data\\USGS_observed.dss',
  'source_dataset': None,
  'title': 'USGS_observed'}]

In [578]:
# open the model application Json template, del unnecessary keys, update, add, export 
with open(r"C:\py\hec_meta_extract\example\input\json\hms_model_application.json", 'r') as f:
            model_template_json = json.load(f)

# keys to drop from json template
drop_keys = ['_id', 'linked_resources', 'common_parameters', 'common_software_version', 'authors', 
'spatial_extent_resolved', 'spatial_valid_extent_resolved', 'temporal_extent', 'temporal_resolution', 
'spatial_valid_extent', 'common_files_details', 'grid']
for key in drop_keys:
    del model_template_json[key]

# set basic keywords
model_template_json['keywords'] = ['hec-hms','hec','hms','hydrology','model','lwi']

model_template_json['purpose'] = kv['Project']['Description']
model_template_json['description'] = kv['Project']['Description']
model_template_json['title'] = f"{kv['Project']['Title']} HEC-HMS Model"

# common_files_details[]
model_template_json['common_files_details'] = []
model_template_json['common_files_details'].append(
    [{
        "description": "The HMS Project File",
        "location": prj_file_tail,
        "source_dataset": None,
        "title": "Project File"
    },
    {
        "description": "There may be multiple basins in the HMS model project",
        "location": f"{prj_dir}/*.basin",
        "source_dataset": None,
        "title": "Basin Files"
    },
    {
        "description": "There may be multiple Meteorological Models",
        "location": f"{prj_dir}/*.met",
        "source_dataset": None,
        "title": "Meteorological Model Files"
    },
    {
        "description": "There may be multiple control specifications.",
        "location": f"{prj_dir}/*.control",
        "source_dataset": None,
        "title": "Control Specification Files"
    }]
)
model_template_json['common_files_details'].extend(dss_common_files_input)
model_template_json['common_files_details']
# Map kv dictionary to the model application json
# application_date, spatial_extent[0], common_files_details[..], purpose, description, title

[[{'description': 'The HMS Project File',
   'location': 'Amite_HMS.hms',
   'source_dataset': None,
   'title': 'Project File'},
  {'description': 'There may be multiple basins in the HMS model project',
   'location': 'Z:/Amite/Amite_LWI/Models/Amite_HMS/*.basin',
   'source_dataset': None,
   'title': 'Basin Files'},
  {'description': 'There may be multiple Meteorological Models',
   'location': 'Z:/Amite/Amite_LWI/Models/Amite_HMS/*.met',
   'source_dataset': None,
   'title': 'Meteorological Model Files'},
  {'description': 'There may be multiple control specifications.',
   'location': 'Z:/Amite/Amite_LWI/Models/Amite_HMS/*.control',
   'source_dataset': None,
   'title': 'Control Specification Files'}],
 {'description': 'User Added from Input DSS File Directory',
  'location': 'Z:\\Amite\\Amite_LWI\\Models\\Amite_HMS\\data\\Amite_HMS_TimeSeries.dss',
  'source_dataset': None,
  'title': 'Amite_HMS_TimeSeries'},
 {'description': 'User Added from Input DSS File Directory',
  'loca

In [579]:
# output model application json
output_prj_json = f'{prj_name}_model_application.json'
with open(output_prj_json, "w") as outfile:
    json.dump(model_template_json, outfile)

In [580]:
# open the simulation Json template, del unnecessary keys, update, add, export

with open(r"C:\py\hec_meta_extract\example\input\json\hms_simulation.json", 'r') as f:
            sim_json = json.load(f)
# model_template_json
# keys to drop from json template
drop_keys = ['_id', 'model_application', 'linked_resources', 'input_files', 'output_files', 'type']
for key in drop_keys:
    del sim_json[key]



# set basic keywords
sim_json['keywords'] = ['hec-hms','hec','hms','hydrology','simulation','lwi']



In [581]:
# Open .run file
with open(os.path.join(prj_dir,f'{prj_name}.run'), 'r') as r:
    run_file = r.readlines()
run_file = [s.strip('\n') for s in run_file]
# run_file

line_start = 0
runList = []
for i,v in enumerate(run_file):
        if v == 'End:':
                # If not the beginning of the file, skip a blank line (+1) for the start of the subList.
                if len(runList) > 0:
                        runList.append(run_file[line_start+1:i])
                else:
                        runList.append(run_file[line_start:i])
                line_start = i+1
runList

[['Run: August 2016',
  '     Default Description: Yes',
  '     Log File: August_2016.log',
  '     DSS File: August_2016.dss',
  '     Is Save Spatial Results: No',
  '     Last Modified Date: 13 January 2023',
  '     Last Modified Time: 16:36:46',
  '     Last Execution Date: 31 January 2023',
  '     Last Execution Time: 20:11:32',
  '     Basin: Consensus (Generic)',
  '     Precip: Specified Hyetographs',
  '     Control: Aug2016',
  '     Time-Series Output: Save All',
  '     Time Series Results Manager Start:',
  '     Time Series Results Manager End:'],
 ['Run: Barry',
  '     Default Description: Yes',
  '     Log File: Barry.log',
  '     DSS File: Barry.dss',
  '     Is Save Spatial Results: No',
  '     Last Modified Date: 13 January 2023',
  '     Last Modified Time: 16:36:46',
  '     Last Execution Date: 25 January 2021',
  '     Last Execution Time: 19:17:16',
  '     Basin: Consensus (Generic)',
  '     Precip: Specified Hyetographs',
  '     Control: Barry',
  '   

In [587]:
sim_kv = {}
for subList in runList:
    title = subList[0].split(":")[1].strip()
    # print(title)
    sim_kv[title] = {}
    findList = ['Basin', 'DSS File', 'Precip', 'Control']
    parameters = {}
    for find_key in findList:
        found_value = [s for s in subList if find_key in s][0].split(":")[1:][0].strip()
        sim_kv[title][find_key] = found_value
        
        # Add data from each simulation's control file.
        if find_key == 'Control':
            control_name = sim_kv[title][find_key].replace(" ","_").replace("(","_").replace(")","_") + '.control'
            control_file = os.path.join(prj_dir, control_name)
            
            with open(control_file, 'r') as c:
                c_file = c.readlines()
            
            c_file  = [s.strip('\n') for s in c_file]
            c_findList = ['Description', 'Start Date', 'End Date', 'Time Interval']
            
            for c_find_key in c_findList:
                found_value = [s for s in c_file if c_find_key in s][0].split(":")[1:][0].strip()
                sim_kv[title][c_find_key] = found_value
        
        # Add data from each simulations's basin file
        parameterList = []
        if find_key == 'Basin':
            basin_name = sim_kv[title][find_key].replace(" ","_").replace("(","_").replace(")","_") + '.basin'
            basin_file = os.path.join(prj_dir, basin_name)
            # print(basin_file)
            print(basin_name)

            with open(basin_file, 'r') as b:
                b_file = b.readlines()
            # print (b_file)
            b_file  = [s.strip('\n') for s in b_file]

            line_start = 0
            basinList = []
            # print (b_file)
            for i,v in enumerate(b_file):
                # print (v + '\n')
                if (v == 'End:'):
                    # print(i, v)
                    # If not the beginning of the file, skip a blank line (+1) for the start of the subList.
                    if len(basinList) > 0:
                            basinList.append(b_file[line_start+1:i])
                    else:
                            basinList.append(b_file[line_start:i])
                    line_start = i+1


            # List of Parameters to look for in each .basin file. Each Parameter will be added as a key to a temporary dictionary before formatting.
            b_findList = ['Canopy', 'LossRate', 'Transform', 'Baseflow', 'Route']
            params = {}
            # initialize empty lists for each parameter key
            for key in b_findList:
                    params[key] = []
            # For each line of each element block in a .basin file, look for each parameter (key) in b_findList 
            for el in basinList:
                for line in el:
                    for key in b_findList:
                        
                        if f'{key}: ' in line:
                            # Append the parameter values to the parameter dictionary
                            params[key].append(line.split(': ')[-1])   
                        
                        # remove duplicates from each key's list of values
                        params[key] = list(set(params[key]))

            # Put parameters dictionary into the required Json format and add to kv dictionary for each run title.    
            for key in params.keys():
                parameterList.append(
                    {
                        "parameter": key,
                        "value": params[key]
                    }
                )
            sim_kv[title]['parameters'] = parameterList

sim_kv



Consensus__Generic_.basin
Consensus__Generic_.basin
Consensus__Generic_.basin
Consensus__Generic_.basin
Consensus__Generic_.basin
Consensus__March_2016_.basin


{'August 2016': {'Basin': 'Consensus (Generic)',
  'parameters': [{'parameter': 'Canopy', 'value': ['Simple']},
   {'parameter': 'LossRate', 'value': ['Deficit Constant']},
   {'parameter': 'Transform', 'value': ['Clark']},
   {'parameter': 'Baseflow', 'value': ['Monthly Constant']},
   {'parameter': 'Route', 'value': ['Muskingum Cunge']}],
  'DSS File': 'August_2016.dss',
  'Precip': 'Specified Hyetographs',
  'Control': 'Aug2016',
  'Description': 'Flood of August 2016',
  'Start Date': '5 August 2016',
  'End Date': '17 August 2016',
  'Time Interval': '60'},
 'Barry': {'Basin': 'Consensus (Generic)',
  'parameters': [{'parameter': 'Canopy', 'value': ['Simple']},
   {'parameter': 'LossRate', 'value': ['Deficit Constant']},
   {'parameter': 'Transform', 'value': ['Clark']},
   {'parameter': 'Baseflow', 'value': ['Monthly Constant']},
   {'parameter': 'Route', 'value': ['Muskingum Cunge']}],
  'DSS File': 'Barry.dss',
  'Precip': 'Specified Hyetographs',
  'Control': 'Barry',
  'Descr

In [583]:
# keyList = ['Canopy', 'Route']
# params = {}

# for key in keyList:
#         params[key] = []

# for el in basinList:
#     for line in el:
#         for key in keyList:
#             if f'{key}: ' in line:
#                 # print (key, line.split(': ')[-1])
#                 params[key].append(line.split(': ')[-1])   
#                 # print (params[key])
#             params[key] = list(set(params[key]))
# params

In [584]:
# open the .gage file
with open(os.path.join(prj_dir,f'{prj_name}.gage'), 'r') as r:
    gage_file = r.readlines()
gage_file = [s.strip('\n') for s in gage_file]
# gage_file

line_start = 0
gageList = []
for i,v in enumerate(gage_file):
        if v == 'End:':
                # If not the beginning of the file, skip a blank line (+1) for the start of the subList.
                if len(gageList) > 0:
                        gageList.append(gage_file[line_start+1:i])
                else:
                        gageList.append(gage_file[line_start:i])
                line_start = i+1
gageList

[['Gage Manager: Amite HMS',
  '     Version: 4.10',
  '     Filepath Separator: \\'],
 ['Gage: Amite',
  '     Last Modified Date: 13 January 2023',
  '     Last Modified Time: 16:49:54',
  '     Reference Height Units: Feet',
  '     Reference Height: 32.81',
  '     Gage Type: Precipitation',
  '     Precipitation Type: Incremental',
  '     Units: IN',
  '     Data Type: PER-CUM',
  '     Data Source Type: Modifiable DSS',
  '     Variant: Variant-1',
  '       Last Variant Modified Date: 22 September 2022',
  '       Last Variant Modified Time: 14:24:55',
  '       Default Variant: Yes',
  '       DSS File Name: rainfall_stageIV.dss',
  '       DSS Pathname: /STAGEIV/AMITEBAYWOOD/PRECIP-INC/19Aug2008 - 12Sep2012/1HOUR/NCEP/',
  '       Start Time: 1 August 2005, 00:00',
  '       End Time: 30 July 2019, 00:00',
  '     End Variant: Variant-1'],
 ['Gage: AmiteBaywood',
  '     Last Modified Date: 31 January 2023',
  '     Last Modified Time: 20:11:15',
  '     Reference Height Unit

In [585]:
# For each gage in .gage file, Get gage type and associated dss file name.
sim_kv['Gage DSS Files'] ={}
for gage in gageList:
    # get gage title
    title = gage[0].split(":")[1].strip()
    # Init ditionary for each gage title
    sim_kv['Gage DSS Files'][title] = {}
    # findList is used to search the wanted .gage file fields for each gage title.
    findList = ["Gage Type", "DSS File Name"]
    # search each gage for the keys in findList, append as key:value pairs for each gage title.
    for find_key in findList:
        found_value = [s for s in gage if find_key in s]
        
        # Omit blank fields by testing the length
        if len(found_value) > 0:
            found_value = found_value[0].split(":")[1:][0].strip()
            
            sim_kv['Gage DSS Files'][title][find_key] = found_value

    # Remove gage titles that did not contain the findList fields.
    if len(sim_kv['Gage DSS Files'][title]) == 0:
        del sim_kv['Gage DSS Files'][title]

# sim_kv['Gage DSS Files']

In [586]:
# Map to Json Template

{'August 2016': {'Basin': 'Consensus (Generic)',
  'DSS File': 'August_2016.dss',
  'Precip': 'Specified Hyetographs',
  'Control': 'Aug2016',
  'Description': 'Flood of August 2016',
  'Start Date': '5 August 2016',
  'End Date': '17 August 2016',
  'Time Interval': '60',
  'parameters': [{'parameter': 'Canopy', 'value': ['Simple']},
   {'parameter': 'LossRate', 'value': ['Deficit Constant']},
   {'parameter': 'Transform', 'value': ['Clark']},
   {'parameter': 'Baseflow', 'value': ['Monthly Constant']},
   {'parameter': 'Route', 'value': ['Muskingum Cunge']}]},
 'Barry': {'Basin': 'Consensus (Generic)',
  'DSS File': 'Barry.dss',
  'Precip': 'Specified Hyetographs',
  'Control': 'Barry',
  'Description': 'Hurricane Barry',
  'Start Date': '2 July 2019',
  'End Date': '25 July 2019',
  'Time Interval': '60',
  'parameters': [{'parameter': 'Canopy', 'value': ['Simple']},
   {'parameter': 'LossRate', 'value': ['Deficit Constant']},
   {'parameter': 'Transform', 'value': ['Clark']},
   {'